In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.animation as animation
import numpy as np
import scipy as sp 

In [2]:
%matplotlib qt

# Animated path on the Bloch sphere

In [4]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set(xlim3d=[-1.2,1.2], ylim3d=[-1.2,1.2], zlim3d=[-1.2,1.2])
npoints = 100

r = 1
u = np.linspace(0,2*np.pi, npoints)
v = np.linspace(0, np.pi, npoints)
x = r * np.outer(np.cos(u), np.sin(v))
y = r * np.outer(np.sin(u), np.sin(v))
z = r * np.outer(np.ones(np.size(u)), np.cos(v))

ax.plot_surface(x,y,z, color='linen', alpha=0.4)

theta = np.linspace(0, 2*np.pi, npoints)
straight = np.zeros(npoints)
round1 = r * np.sin(theta)
round2 = r * np.cos(theta)

ax.plot(round1, round2, straight, color='black', alpha=0.75)
ax.plot(straight, round1, round2, color='black', alpha=0.75)
#ax.plot(round1, straight, round2, color='black', alpha=0.75)

line = np.linspace(-r,r,npoints)

ax.plot(line, straight, straight, color='black', alpha=0.6)
ax.plot(straight, line, straight, color='black', alpha=0.6)
ax.plot(straight, straight, line, color='black', alpha=0.6)

ar = ax.quiver(
        0, 0, 0, # <-- starting point of vector
        0, 0, r, # <-- directions of vector
        color = 'magenta', alpha = 1, lw = 3,
        arrow_length_ratio = 0.15
    )

nperiods = 4
nframes = 400
data3d = np.zeros((nframes, 3))
data3d[0] = np.array([0,0,r])

sc = ax.scatter(0,0,r, c='red', s=2)
phi = 0
def update(frame):
    global ar
    global phi
    ar.remove()
    theta = nperiods * 2 * np.pi * frame / nframes
    phi += np.random.normal(0,0.03)
    tipx = r * np.sin(phi) * np.sin(theta)
    tipy = r * np.cos(phi) * np.sin(theta)
    tipz = r * np.cos(theta)
    ar = ax.quiver(0, 0, 0, # <-- starting point of vector
        tipx, tipy, tipz, # <-- directions of vector
        color = 'magenta', alpha = 1, lw = 3,
        arrow_length_ratio = 0.15)
    
    data3d[frame] = np.array([tipx,tipy,tipz])
    sc._offsets3d = (data3d[:frame,0], data3d[:frame,1], data3d[:frame,2])
    return (sc,ar,)
     
ani = animation.FuncAnimation(fig=fig, func=update, frames=nframes, interval=30)
ani.save(filename="anim.mp4", writer="ffmpeg")

plt.show()

# Control scheme for a two-level system

In [ ]:
rho_x = np.array([[0,1],[1,0]])
rho_y = np.array([[0,-1j],[1j,0]])
rho_z = np.array([[1,0],[0,-1]])

rho_p = np.array([[0,1],[0,0]])
rho_m = np.array([[0,0],[1,0]])

psi_0 = np.array([1,0])
psi_1 = np.array([0,1])

def H(w, g):
    return w * rho_z + g * rho_x

rho = np.outer(np.conj(psi_0), psi_0)

In [7]:
def unitary_step(H, state, dt):
    U = sp.linalg.expm(-1j * H * dt)
    new_state = U @ state
    return new_state

In [8]:
T = 3
nsteps = 100

dt = T/nsteps

sigma_z = np.zeros(nsteps, dtype='complex')
sigma_x = np.zeros(nsteps, dtype='complex')

state = (psi_0 + psi_1 ) / np.sqrt(2)
target = psi_1

ts = np.linspace(0,T,nsteps)
fidelities = np.zeros(nsteps)
work = 0

# easy case:
state = psi_0
target = psi_1
# from theory we get:
w_opt = 0
g_opt = np.pi/(2*T)
H_tmp = H(w_opt, g_opt)

# harder case:
#state = (psi_0 + psi_1 ) / np.sqrt(2)
#target = psi_1

for i in range(nsteps):
    #H_tmp = H(2.2*np.sin(i*dt), np.cos(1.1*i*dt))

    sigma_x[i] = np.dot(np.conj(state), rho_x @ state)
    sigma_z[i] = np.dot(np.conj(state), rho_z @ state)
    work += np.dot(np.conj(state), H_tmp @ state)*dt
    fidelities[i] = np.abs(np.dot(np.conj(target), state))**2
    state = unitary_step(H_tmp, state, dt)



In [9]:
fig, ax = plt.subplots()

# turns out in most cases the imaginary parts are zero.

ax.plot(ts, np.real(sigma_x), c='cyan', label=r'$\sigma_x$')
#ax.plot(ts, np.imag(sigma_x), c='cyan', ls=':')

ax.plot(ts, np.real(sigma_z), c='pink', label=r'$\sigma_z$')
#ax.plot(ts, np.imag(sigma_z), c='pink', ls=":")

ax.plot(ts, fidelities, c='black', ls='--', label='F')

ax.legend(loc='best')

fig.suptitle(f'W={work.real:.4}')
plt.show()

# Function for evolving the state given the discretized amplitudes

In [10]:
def evolution(initial_state, ws, gs, nsteps, dt, return_measurements=False, return_fidelity=False, target=None):
    """The state is evolved unitarily over intervals of length dt with constant parameter values given by the tables ws, gs"""
    sigma_z = np.zeros(nsteps, dtype='complex')
    sigma_x = np.zeros(nsteps, dtype='complex')

    fidelities = np.zeros(nsteps)
    state = initial_state
    H_t0 = H(ws[0], gs[0])
    sigma_x[0] = np.dot(np.conj(state), rho_x @ state)
    sigma_z[0] = np.dot(np.conj(state), rho_z @ state)
    fidelity = np.abs(np.dot(np.conj(target), state))**2
    fidelities[0] = fidelity
    work = 0 
    for i in range(1,nsteps):
        state = unitary_step(H_t0, state, dt)
        sigma_x[i] = np.dot(np.conj(state), rho_x @ state)
        sigma_z[i] = np.dot(np.conj(state), rho_z @ state)
        fidelity = np.abs(np.dot(np.conj(target), state))**2
        fidelities[i] = fidelity
        H_t1 = H(ws[i], gs[i])
        dH = H_t1 - H_t0
        dU = np.dot(np.conj(state), dH @ state)
        work += dU # we identify work with the change of internal energy

        H_t0 = H_t1
        
    if return_measurements:
        return state, sigma_x, sigma_z, fidelities, work.real
    elif return_fidelity:
        return state, fidelity
    else:
        return state
    
    

In [11]:
nsteps = 100
T = 0.5
dt = T/nsteps

ws = np.ones(nsteps) * 2 * np.pi / (T) 
gs = np.ones(nsteps) * 1 * np.pi / (T)
ws[:int(nsteps/2)] = 1
gs[:int(nsteps/2)] = 2

psi_init = (psi_0 + psi_1) / np.sqrt(2)
psi_final = (psi_0 - psi_1) / np.sqrt(2)

state, sigma_x, sigma_z, fidelities, work = evolution(psi_init, ws, gs, nsteps, dt, return_measurements=True, target=psi_final)

In [16]:
fig, ax = plt.subplots()

# turns out in most cases the imaginary parts are zero.

ax.plot(ts, np.real(sigma_x), c='cyan', label=r'$\sigma_x$')
#ax.plot(ts, np.imag(sigma_x), c='cyan', ls=':')

ax.plot(ts, np.real(sigma_z), c='pink', label=r'$\sigma_z$')
#ax.plot(ts, np.imag(sigma_z), c='pink', ls=":")

ax.plot(ts, fidelities, c='black', ls='--', label='F')

ax.legend(loc='best')

fig.suptitle(f'W={work.real:.4}')
plt.show()

# Optimizing the control for a selected target state

In [17]:
from scipy.optimize import minimize

In [22]:
def objective(params, state, target, nsteps, dt):
    """
    The argument params is a numpy array of shape (1, 2*nsteps). First nsteps values contain the ws, second nsteps values contain the gs.
    The objective is maximizing the fidelity of the final state, which we turn into the minimization of a function 1-fidelity.
    """
    ws = params[:nsteps]
    gs = params[nsteps:]
    state, sigma_x, sigma_z, fidelities, work = evolution(state, ws, gs, nsteps, dt, return_measurements=True, target=target)
    fid = fidelities[-1]
    return  work**2 + (1 - fid) 

In [30]:
nsteps = 10
T = 1
dt = T/nsteps
epsilon = 0.001

# stating the problem
psi_init = (psi_0 + psi_1) / np.sqrt(2)
#psi_final = (psi_0 - psi_1) / np.sqrt(2)

# different problem
psi_init = psi_0
#psi_final = psi_1

# different problem
psi_init = psi_0
v = 2 * np.pi
u = 0
#psi_final = np.cos(v/2) * psi_0 + np.exp(1j * u)*np.sin(v/2) * psi_1

# zero initial parameters:
params = np.zeros(2*nsteps) + epsilon

# random initial parameters:
params = np.random.rand(2*nsteps)

# tailored initial parameters (from theory we know they should be constant) with some deviation epsilon
#params = np.append(np.zeros(nsteps)+epsilon, np.ones(nsteps)*np.pi/(2*T)-epsilon)

sol = minimize(objective, params, args=(psi_init, psi_final, nsteps, dt))

In [31]:
params_opt = sol['x']

ws_opt = params_opt[:nsteps]
gs_opt = params_opt[nsteps:]

state, sigma_x, sigma_z, fidelities, work = evolution(psi_init, ws_opt, gs_opt, nsteps, dt, return_measurements=True, target=psi_final)

In [32]:
print("target: " + str(psi_1))
print("result: " + str(state))
print("rounded: "+ str(state.round(1)))
print("absolute value: "+ str(np.abs(state).round(2)))
print("fidelity: " + str(fidelities[-1]))

target: [0 1]
result: [ 0.35359166+0.61234916j -0.35359205-0.6123513j ]
rounded: [ 0.4+0.6j -0.4-0.6j]
absolute value: [0.71 0.71]
fidelity: 0.9999999999976319


In [33]:
fig, ax = plt.subplots()
ts = np.linspace(0,T,nsteps)
ax.plot(ts, ws_opt, label=r'$\omega$')
ax.plot(ts, gs_opt, label=r'g')
ax.legend(loc='best')
ax.hlines([np.mean(gs_opt), np.mean(ws_opt)], 0, T, colors='black', ls='--')
plt.show()

In [34]:
fig, ax = plt.subplots()

# turns out in most cases the imaginary parts are zero.

ax.plot(ts, np.real(sigma_x), c='cyan', label=r'$\sigma_x$')
#ax.plot(ts, np.imag(sigma_x), c='cyan', ls=':')

ax.plot(ts, np.real(sigma_z), c='pink', label=r'$\sigma_z$')
#ax.plot(ts, np.imag(sigma_z), c='pink', ls=":")

ax.plot(ts, fidelities, c='black', ls='--', label='F')

ax.legend(loc='best')

fig.suptitle(f'W={work.real:.4}, F={fidelities[-1]:.4}')
plt.show()

# Fidelities for a given final state visualised on the Bloch sphere

In [35]:
nthetas = 5
nphis = 5

# we add one point and then delete it to avoid overlapping the first and last values
thetas = np.linspace(0,2*np.pi, nthetas+1)[:-1]
phis = np.linspace(0, np.pi, nphis)

print(thetas)
print(phis)

[0.         1.25663706 2.51327412 3.76991118 5.02654825]
[0.         0.78539816 1.57079633 2.35619449 3.14159265]


In [46]:
from matplotlib import cm

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set(xlim3d=[-1.2,1.2], ylim3d=[-1.2,1.2], zlim3d=[-1.2,1.2])
npoints = 16

r = 1
u = np.linspace(0,2*np.pi, npoints+1)[:-1] # phis - the one in the exponent in the bloch form; we add and omit the last element to avoid overlapping 0 and 2*pi
v = np.linspace(0, np.pi, npoints) # thetas - the one in cosine giving z in the polar form
x = r * np.outer(np.cos(u), np.sin(v))
y = r * np.outer(np.sin(u), np.sin(v))
z = r * np.outer(np.ones(np.size(u)), np.cos(v))

nsteps = 3
T = 1
dt = T/nsteps
epsilon = 0.001

# here we fix the initial state
# psi_init = (psi_0 + 1j*psi_1) / np.sqrt(2)
psi_init = psi_0

# fixing initial state with the angles:
v_init = 0*np.pi/2 # polar, range: [0, pi]
u_init = 0*np.pi/2 # azimuthal, range: [0, 2*pi]
psi_init = np.cos(v_init/2) * psi_0 + np.exp(1j * u_init)*np.sin(v_init/2) * psi_1

# random initial parameters:
params = np.random.rand(2*nsteps)

# array for storing the obtained fidelities
of = np.ones_like(z) 

for i in range(npoints):
    for j in range(npoints):
        psi_final = np.cos(v[j]/2) * psi_0 + np.exp(1j * u[i])*np.sin(v[j]/2) * psi_1
        #print(psi_final)
        sol = minimize(objective, params, args=(psi_init, psi_final, nsteps, dt))
        params_opt = sol['x']
        ws_opt = params_opt[:nsteps]
        gs_opt = params_opt[nsteps:]
        state, sigma_x, sigma_z, fidelities, work = evolution(psi_init, ws_opt, gs_opt, nsteps, dt, return_fidelity=False, return_measurements=True, target=psi_final)
        fidelity = fidelities[-1]
        #print(fidelity)
        of[i,j] = fidelity
        if np.abs(1-fidelity) > 0.001:
            pass
            #print(psi_final)
            #print(f"[i,j]=[{i},{j}]; [u,v]=[{u[i]:.3},{v[j]:.3}]; F={fidelity:.3}; target: " + str(psi_final.round(3)) + "; state: " + str(state.round(3)))

#ax.plot_surface(x,y,z, alpha=1, facecolors=cm.jet(of/np.amax(of)))
#ax.scatter(x,y,z, c=cm.jet(of/np.amax(of)))

import matplotlib.colors as mcolors
norm = mcolors.Normalize(vmin=0, vmax=1)
sc = ax.scatter(x,y,z, c=of, alpha=0.8, s=50, cmap='plasma',norm=norm)
#cbar = fig.colorbar(sc)

mappable = cm.ScalarMappable(norm=norm, cmap='plasma')
cbar = fig.colorbar(mappable, ax=ax)

theta = np.linspace(0, 2*np.pi, npoints)
straight = np.zeros(npoints)
round1 = r * np.sin(theta)
round2 = r * np.cos(theta)

ax.plot(round1, round2, straight, color='black', alpha=0.8)
ax.plot(straight, round1, round2, color='black', alpha=0.8)
#ax.plot(round1, straight, round2, color='black', alpha=0.75)

line = np.linspace(-r,r,npoints)

ax.plot(line, straight, straight, color='black', alpha=0.9)
ax.plot(straight, line, straight, color='black', alpha=0.9)
ax.plot(straight, straight, line, color='black', alpha=0.9)

tt = np.pi/2
pp = np.pi/2


# draw the initial state
arx = np.cos(u_init) * np.sin(v_init)
ary = np.sin(u_init) * np.sin(v_init)
arz = np.cos(v_init)
ax.quiver(
                0, 0, 0, # <-- starting point of vector
                arx, ary, arz, # <-- directions of vector
                color = 'red', alpha = 1, lw = 2, # alpha=0 makes it invisible
                arrow_length_ratio = 0.15
            )


ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

ax.set_box_aspect((np.ptp(x), np.ptp(y), np.ptp(z)))
fig.suptitle('Fidelities for different target states')
plt.show()

In [47]:
print(f'F_min = {np.min(of):.5}')
print(f'F_max = {np.max(of):.5}')

F_min = 1.0
F_max = 1.0


# Finding the control scheme with ML

In [48]:
import jax, flax